In [ ]:
""" 2 Finding edges """

import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['image.cmap'] = 'gray'
import pdb

# load image
img = io.imread('bird.jpg')

# copy functions myconv2, gauss1d, gauss2d and gconv from exercise 1
def myconv2(image, filt):
    # This function performs a 2D convolution between image and filt, image being a 2D image. This
    # function should return the result of a 2D convolution of these two images. DO
    # NOT USE THE BUILT IN SCIPY CONVOLVE within this function. You should code your own version of the
    # convolution, valid for both 2D and 1D filters.
    # INPUTS
    # @ image         : 2D image, as numpy array, size mxn
    # @ filt          : 1D or 2D filter of size kxl
    # OUTPUTS
    # img_filtered    : 2D filtered image, of size (m+k-1)x(n+l-1)

    # if filt is 1D of shape (length, None) (this is common in python), transform it to shape (1, length)
    if filt.ndim < 2:
        filt = filt[np.newaxis, :]
    if image.ndim < 2:
        image = image[np.newaxis, :]
    # flip the filter
    filt = np.flipud(np.fliplr(filt))
    # initialize the filtered image
    filtered_img = np.empty((image.shape[0] + filt.shape[0] - 1, image.shape[1] + filt.shape[1] - 1))
    # pad original image with zeros outside of the borders
    padded_img = np.pad(image, ((filt.shape[0] - 1,), (filt.shape[1] - 1, )), 'constant')
    # calculate each element of the full filtered image
    for index in np.ndindex(filtered_img.shape):
        filtered_img[index] = np.sum(padded_img[index[0]:index[0]+filt.shape[0], index[1]:index[1]+filt.shape[1]] * filt)

    return filtered_img


def gauss1d(sigma, filter_length=3):
    # INPUTS
    # @ sigma         : sigma of gaussian distribution
    # @ filter_length : integer denoting the filter length, default is 10
    # OUTPUTS
    # @ gauss_filter  : 1D gaussian filter

    # if filter_lenght is even add one
    filter_length += ~filter_length % 2
    x = np.linspace(-filter_length/2, filter_length/2, filter_length)

    gauss_filter = np.exp(- (x ** 2) / (2 * (sigma ** 2)))

    gauss_filter = gauss_filter / np.sum(gauss_filter)
    return gauss_filter


def gauss2d(sigma, filter_size=3):
    # INPUTS
    # @ sigma         : sigma of gaussian distribution
    # @ filter_size   : integer denoting the filter size, default is 10
    # OUTPUTS
    # @ gauss2d_filter  : 2D gaussian filter

    # create a 1D gaussian filter
    gauss1d_filter = gauss1d(sigma, filter_size)[np.newaxis, :]
    # convolve it with its transpose
    gauss2d_filter = myconv2(gauss1d_filter, np.transpose(gauss1d_filter))
    return gauss2d_filter


def gconv(image, sigma):
    # INPUTS
    # image           : 2d image
    # @ sigma         : sigma of gaussian distribution
    # OUTPUTS
    # @ img_filtered  : filtered image with gaussian filter

    gauss2d_filter = gauss2d(sigma)
    img_filtered = myconv2(image, gauss2d_filter)

    return img_filtered


# 2.1
# Gradients
# define a derivative operator
dx = np.expand_dims(np.array((-1, 0, 1)), axis=0)
print(dx.shape)
dy = np.transpose(dx)
print(dy.shape)

# convolve derivative operator with a 1d gaussian filter with sigma = 1
# You should end up with 2 1d edge filters,  one identifying edges in the x direction, and
# the other in the y direction
sigma = 1
g1d_filter = gauss1d(sigma)
gdx = myconv2(g1d_filter, dx)
gdy = np.transpose(gdx)

print(gdx)


# 2.2
# Gradient Edge Magnitude Map
def create_edge_magn_image(image, dx, dy):
    # this function created an eddge magnitude map of an image
    # for every pixel in the image, it assigns the magnitude of gradients
    # INPUTS
    # @image  : a 2D image
    # @dx     : gradient along x axis
    # @dy     : geadient along y axis
    # OUTPUTS
    # @grad_mag_image   : 2d image same size as image, with the magnitude of gradients in every pixel
    # @grad_dir_image   : 2d image same size as image, with the direcrion of gradients in every pixel

    img_grad_x = myconv2(image, dx)
    img_grad_y = myconv2(image, dy)

    # myconv2 returns the full convolution, so we should take the central part of the image, of size
    # equal to image.shape
    grad_x_crop_x = int((img_grad_x.shape[0] - image.shape[0])/2)
    grad_x_crop_y = int((img_grad_x.shape[1] - image.shape[1])/2)
    grad_y_crop_x = int((img_grad_y.shape[0] - image.shape[0])/2)
    grad_y_crop_y = int((img_grad_y.shape[1] - image.shape[1])/2)
    img_grad_x = img_grad_x[grad_x_crop_x:img_grad_x.shape[0]-grad_x_crop_x, grad_x_crop_y:img_grad_x.shape[1]-grad_x_crop_y]
    img_grad_y = img_grad_y[grad_y_crop_x:img_grad_y.shape[0]-grad_y_crop_x, grad_y_crop_y:img_grad_y.shape[1]-grad_y_crop_y]

    grad_mag_image = np.sqrt((img_grad_x ** 2 + img_grad_y ** 2))   # Magnitude
    grad_dir_image = np.arctan2(img_grad_y, img_grad_x)      # Orientation in rads [-pi, pi]

    return grad_mag_image, grad_dir_image


# create an edge magnitude image using the derivative operators
img_edge_mag, img_edge_dir = create_edge_magn_image(img, dx, dy)

# show all together
plt.subplot(121)
plt.imshow(img)
plt.axis('off')
plt.title('Original image')
plt.subplot(122)
plt.imshow(img_edge_mag)
plt.axis('off')
plt.title('Edge magnitude map')
plt.show()

# 2.3
# Edge images of particular directions
def make_edge_map(image, dx, dy):
    # INPUTS
    # @image        : a 2D image
    # @dx           : gradient along x axis
    # @dy           : geadient along y axis
    # OUTPUTS:
    # @ edge maps   : a 3D array of shape (image.shape, 8) containing the edge maps on 8 orientations

    grad_mag_img, grad_dir_img = create_edge_magn_image(image, dx, dy)
    # initialize edge maps
    edge_maps = np.zeros((image.shape[0], image.shape[1], 4))
    # iterate over different orientations
    for m in range(4):
        # define the angle limits for this edge map
        lim1 = 2 * m * np.pi/4 - np.pi/4
        lim2 = 2 * m * np.pi/4 + np.pi/4

        # remap limits to [-pi, pi], because this is the interval that arctan2 returns
		# this can be done in various ways
        lim1 = np.arctan2(np.sin(lim1), np.cos(lim1))
        lim2 = np.arctan2(np.sin(lim2), np.cos(lim2))

        if m == 2:  # this is the only case that lim1>lim2
            edge_maps[:, :, m] = ((grad_mag_img > 40) & ((lim1 < grad_dir_img) | (lim2 > grad_dir_img)))
        else:
            edge_maps[:, :, m] = ((grad_mag_img > 40) & (lim1 < grad_dir_img) & (lim2 > grad_dir_img))

    edge_maps[edge_maps == 1] = 255

    return edge_maps


# verify with circle image
circle = plt.imread('circle.jpg')
# circle = color.rgb2gray(circle)
edge_maps = make_edge_map(circle, dx, dy)
edge_maps_in_row = [edge_maps[:, :, m] for m in range(edge_maps.shape[2])]
all_in_row = np.concatenate((edge_maps_in_row), axis=1)
plt.imshow(np.concatenate((circle, all_in_row), axis=1))
plt.title('Circle and edge orientations')
# plt.imshow(np.concatenate(np.dsplit(edge_maps, edge_maps.shape[2]), axis=0))
plt.show()

# now try with original image
edge_maps = make_edge_map(img, dx, dy)
edge_maps_in_row = [edge_maps[:, :, m] for m in range(edge_maps.shape[2])]
all_in_row = np.concatenate((edge_maps_in_row), axis=1)
plt.imshow(np.concatenate((img, all_in_row), axis=1))
plt.title('Original image and edge orientations')
plt.show()


(1, 3)
(3, 1)
[[-0.19684199 -0.60631602  0.          0.60631602  0.19684199]]
